<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4ee5207ccba560fe5bb63a7aa25461ae7a25a7165f0654b7f25a48279ce72d02
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-16 14:46:42
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -13.38 K (-0.1%)
Current PnL: -28.93 L (-18.45%)
CY Booked + Current PnL: -14.64 L (-9.34%)
-------------------
Total profit:  1.88 L
Total loss:  -30.81 L
-------------------
Total Booked + Current PnL: 12.34 L (9.5%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.2%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.39 L (67.35%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.12%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.42,15.72,4.91,21.40,15081.0,41722.0,307148.0,7.34,62.0,X-LC,6.59,14.0,2.77,2.19,23.04,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.74,5.50,13.01,19.23,20651.0,8270.0,158730.0,24.27,66.0,L-LC,9.22,182.0,0.40,1.13,27.94,XY25,ATH,MINING
77,TTKPRESTIG,770.00,0.83,-22.41,29.00,0.09,22676.0,-22583.0,78194.0,81.43,36.0,M-SC,3.86,253.0,-1.00,0.56,2.73,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,1.11,-13.80,16.04,0.02,24134.0,-24097.0,150462.0,129.35,54.0,M-SC,9.09,240.0,-1.00,1.07,30.88,OX40N,NTT,PAINTS
49,MASFIN,398.61,0.45,-4.30,27.53,22.05,25814.0,-4215.0,93765.0,-17.34,46.0,H-SC,5.55,168.0,-0.16,0.67,36.29,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,8.16,-1.72,110.35,106.72,133756.0,-2125.0,121211.0,-51.39,58.0,H-SC,8.09,138.0,-0.02,0.87,21.25,AR,ATH,MISC
41,INFY,1972.00,5.18,14.55,17.19,34.24,61612.0,45527.0,358415.0,-10.62,64.0,X-LC,7.69,6.0,0.74,2.56,24.04,X40,NTT,IT
55,RAJESHEXPO,518.00,5.00,-66.52,198.87,0.07,91341.0,-91247.0,45930.0,1583.08,42.0,L-SC,0.75,269.0,-1.00,0.33,14.23,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,3.34,-21.39,91.81,50.79,101512.0,-30085.0,110568.0,0.89,54.0,H-SC,10.17,171.0,-0.30,0.79,28.74,XR,ATH,IT
18,COFFEEDAY,80.00,2.91,-42.02,133.30,35.27,87761.0,-47712.0,65837.0,-55.73,46.0,L-SC,4.72,268.0,-0.54,0.47,61.14,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-3.87,-47.48,318.32,119.69,452645.0,-128563.0,142198.0,-47.02,25.0,H-SC,12.58,150.0,-0.28,1.01,12.34,AR,ATH,IT
22,DIXON,18931.72,-2.98,-25.31,75.75,31.26,130557.0,-58416.0,172352.0,-65.15,22.0,X-MC,21.55,56.0,-0.45,1.23,0.00,X40N,ATH,IT
80,VALIANTORG,838.00,-2.63,-48.09,235.47,74.15,199989.0,-78673.0,84932.0,-252.84,34.0,H-SC,22.08,145.0,-0.39,0.61,8.87,XR,NTT,CHEMICALS
45,JIOFIN,387.00,-2.33,-9.84,38.07,24.48,91771.0,-26307.0,241058.0,-11.32,34.0,X-LC,1.70,37.0,-0.29,1.72,39.72,XY24,BTT,FINANCE
5,ASIANPAINT,3460.25,-2.14,-3.75,25.65,20.95,62159.0,-9434.0,242334.0,-4.71,44.0,X-LC,12.77,36.0,-0.15,1.73,30.55,X40,ATH,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.53,0.66,19.94,20.74,40781.0,1348.0,204518.0,33.49,57.0,M-MC,5.18,189.0,0.03,1.46,11.35,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.56,-1.05,110.12,107.91,160879.0,-1550.0,146094.0,-19.34,56.0,M-SC,9.95,220.0,-0.01,1.04,8.93,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.11,-13.80,16.04,0.02,24134.0,-24097.0,150462.0,129.35,54.0,M-SC,9.09,240.0,-1.00,1.07,30.88,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,2.40,-18.76,62.52,32.03,66961.0,-24732.0,107103.0,-12.01,58.0,H-SC,6.55,157.0,-0.37,0.76,19.96,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.83,-22.41,29.00,0.09,22676.0,-22583.0,78194.0,81.43,36.0,M-SC,3.86,253.0,-1.00,0.56,2.73,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.00,-22.63,46.67,13.47,97367.0,-61038.0,208629.0,-68.12,47.0,H-SC,3.96,159.0,-0.63,1.49,6.08,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.53,0.66,19.94,20.74,40781.0,1348.0,204518.0,33.49,57.0,M-MC,5.18,189.0,0.03,1.46,11.35,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.74,5.50,13.01,19.23,20651.0,8270.0,158730.0,24.27,66.0,L-LC,9.22,182.0,0.40,1.13,27.94,XY25,ATH,MINING
84,WIPRO,420.00,2.84,9.76,56.95,72.27,105917.0,16534.0,185982.0,-5.63,56.0,M-LC,6.35,101.0,0.16,1.33,16.54,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,2.84,9.76,56.95,72.27,105917.0,16534.0,185982.0,-5.63,56.0,M-LC,6.35,101.0,0.16,1.33,16.54,XR,NTT,IT
25,FINCABLES,1641.55,0.56,-1.05,110.12,107.91,160879.0,-1550.0,146094.0,-19.34,56.0,M-SC,9.95,220.0,-0.01,1.04,8.93,OX40N,ATH,CABLES
38,INDIAMART,4810.62,8.16,-1.72,110.35,106.72,133756.0,-2125.0,121211.0,-51.39,58.0,H-SC,8.09,138.0,-0.02,0.87,21.25,AR,ATH,MISC
37,IEX,219.00,0.15,-5.27,56.90,48.63,107774.0,-10544.0,189410.0,-36.95,49.0,H-SC,13.22,137.0,-0.10,1.35,5.73,XR,NTT,MISC
49,MASFIN,398.61,0.45,-4.30,27.53,22.05,25814.0,-4215.0,93765.0,-17.34,46.0,H-SC,5.55,168.0,-0.16,0.67,36.29,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.57,-19.24,37.18,10.78,72157.0,-46242.0,194076.0,-51.78,12.0,X-LC,9.35,1.0,-0.64,1.39,0.00,X40,NTT,FMCG
7,AWL,485.00,-2.01,-30.79,127.12,57.19,286925.0,-100415.0,225712.0,-67.33,18.0,X-MC,11.01,58.0,-0.35,1.61,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-2.98,-25.31,75.75,31.26,130557.0,-58416.0,172352.0,-65.15,22.0,X-MC,21.55,56.0,-0.45,1.23,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.03,-41.60,131.50,35.20,98820.0,-53522.0,75148.0,-1.52,26.0,X-SC,17.60,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,-1.94,-12.12,32.48,16.43,63304.0,-26871.0,194903.0,-25.59,29.0,X-MC,7.45,66.0,-0.42,1.39,15.02,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JIOFIN,387.00,-2.33,-9.84,38.07,24.48,91771.0,-26307.0,241058.0,-11.32,34.0,X-LC,1.70,37.0,-0.29,1.72,39.72,XY24,BTT,FINANCE
73,TCS,4389.97,0.51,-11.77,36.81,20.71,122845.0,-44511.0,333726.0,-23.95,48.0,X-LC,1.89,3.0,-0.36,2.38,11.51,X40,ATH,IT
78,UNITDSPR,1644.00,0.73,-1.14,22.16,20.77,51296.0,-2666.0,231478.0,-9.72,38.0,X-MC,1.97,70.0,-0.05,1.65,5.70,X40N,NTT,BREWERIES
20,DABUR,735.00,0.15,0.59,42.86,43.70,106949.0,1464.0,249532.0,-7.78,55.0,X-MC,2.14,73.0,0.01,1.78,15.73,XY24,BTT,FMCG
65,SIEMENS,4671.50,-1.71,-21.55,59.98,25.51,87571.0,-40095.0,146000.0,-10.47,34.0,H-LC,2.50,50.0,-0.46,1.04,7.59,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-2.01,-30.79,127.12,57.19,286925.0,-100415.0,225712.0,-67.33,18.0,X-MC,11.01,58.0,-0.35,1.61,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.03,-41.60,131.50,35.20,98820.0,-53522.0,75148.0,-1.52,26.0,X-SC,17.60,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
9,BAJAJHLDNG,14451.52,-0.40,-4.34,35.09,29.22,63817.0,-8259.0,181866.0,-6.67,35.0,X-LC,11.98,29.0,-0.13,1.30,0.0,X40,ATH,FINANCE
8,BAJAJHFL,181.50,-1.11,-20.81,96.32,55.47,185576.0,-50620.0,192666.0,-29.75,31.0,X-MC,15.92,63.0,-0.27,1.37,0.0,X40N,ATH,FINANCE
22,DIXON,18931.72,-2.98,-25.31,75.75,31.26,130557.0,-58416.0,172352.0,-65.15,22.0,X-MC,21.55,56.0,-0.45,1.23,0.0,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.08,-30.26,104.83,42.84,47523.0,-19673.0,45333.0,-54.27,42.0,X-SC,24.88,83.0,-0.41,0.32,2.56,XY24,NTT,MISC
58,RELAXO,1176.00,-1.32,-49.76,203.17,52.31,148162.0,-72235.0,72925.0,-46.47,31.0,X-SC,8.07,92.0,-0.49,0.52,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.03,-41.60,131.50,35.20,98820.0,-53522.0,75148.0,-1.52,26.0,X-SC,17.60,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.21,-4.10,29.39,24.08,35780.0,-5209.0,121741.0,-11.91,41.0,X-SC,5.88,91.0,-0.15,0.87,14.71,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.24,-17.41,73.53,43.32,98913.0,-28352.0,134520.0,-29.05,47.0,X-SC,8.17,90.0,-0.29,0.96,3.29,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.57,-19.24,37.18,10.78,72157.0,-46242.0,194076.0,-51.78,12.0,X-LC,9.35,1.0,-0.64,1.39,0.00,X40,NTT,FMCG
75,TMPV,600.00,1.44,-27.27,69.09,22.98,113512.0,-61602.0,164296.0,-24.30,45.0,X-LC,3.64,2.0,-0.54,1.17,3.33,XY24,NTT,AUTO
73,TCS,4389.97,0.51,-11.77,36.81,20.71,122845.0,-44511.0,333726.0,-23.95,48.0,X-LC,1.89,3.0,-0.36,2.38,11.51,X40,ATH,IT
81,VBL,671.64,-0.23,1.25,33.99,35.67,108699.0,3956.0,319798.0,-10.77,60.0,X-LC,2.78,5.0,0.04,2.28,15.28,X40N,ATH,FMCG
41,INFY,1972.00,5.18,14.55,17.19,34.24,61612.0,45527.0,358415.0,-10.62,64.0,X-LC,7.69,6.0,0.74,2.56,24.04,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.26,-30.08,90.07,32.89,48619.0,-23228.0,53979.0,-766.44,73.0,L-MC,9.17,259.0,-0.48,0.39,48.65,XR,NTT,BANKS
13,BSOFT,831.70,3.34,-21.39,91.81,50.79,101512.0,-30085.0,110568.0,0.89,54.0,H-SC,10.17,171.0,-0.30,0.79,28.74,XR,ATH,IT
69,SURYODAY,216.00,1.05,-19.13,52.14,23.03,75503.0,-34262.0,144809.0,57.89,52.0,H-SC,4.45,165.0,-0.45,1.03,43.68,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,1.11,-13.80,16.04,0.02,24134.0,-24097.0,150462.0,129.35,54.0,M-SC,9.09,240.0,-1.00,1.07,30.88,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.74,5.50,13.01,19.23,20651.0,8270.0,158730.0,24.27,66.0,L-LC,9.22,182.0,0.40,1.13,27.94,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.08,-27.96,56.73,12.90,89290.0,-61102.0,157394.0,131.97,61.0,H-SC,8.97,173.0,-0.68,1.12,30.87,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,2.40,-18.76,62.52,32.03,66961.0,-24732.0,107103.0,-12.01,58.0,H-SC,6.55,157.0,-0.37,0.76,19.96,OX40N,NTT,IT
17,COALINDIA,484.83,-0.74,5.50,13.01,19.23,20651.0,8270.0,158730.0,24.27,66.0,L-LC,9.22,182.0,0.40,1.13,27.94,XY25,ATH,MINING
40,INDUSINDBK,1800.00,0.26,-30.08,90.07,32.89,48619.0,-23228.0,53979.0,-766.44,73.0,L-MC,9.17,259.0,-0.48,0.39,48.65,XR,NTT,BANKS
57,RECLTD,446.00,0.53,0.66,19.94,20.74,40781.0,1348.0,204518.0,33.49,57.0,M-MC,5.18,189.0,0.03,1.46,11.35,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.49
1,25,44.55
2,50,76.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.82
MC    30.43
LC    26.75
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.29
X40      24.05
X40N     12.73
XY25     10.81
XR        9.01
AR        8.41
OX40N     7.75
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.07
X-LC    23.25
H-SC    22.89
M-SC    11.25
X-SC     7.88
H-MC     4.51
M-MC     1.46
M-LC     1.33
L-LC     1.13
H-LC     1.04
L-SC     0.80
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.15,-9.54,45.48
IT,12.91,-18.11,77.68
FINANCE,11.11,-14.57,62.66
MISC,7.13,-30.20,83.46
ELECTRICAL,5.81,-15.29,57.51
PAINTS,5.78,-13.87,30.78
INSURANCE,4.70,-2.90,38.11
PHARMA,3.91,-7.40,41.21
AUTO,2.81,-33.11,79.26


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3287622.0,21
AR,1394515.0,10
XR,1230905.0,12
X40,1152159.0,15
X40N,957344.0,9
OX40N,704554.0,10
XY25,425799.0,7
SR,286123.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3585455.0,24
X-MC,1658179.0,16
M-SC,1484593.0,15
X-LC,965785.0,13
X-SC,815838.0,8
H-MC,446530.0,3
L-SC,179102.0,2
M-LC,105917.0,1
H-LC,87571.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1261165.0      6
           AR         936211.0      5
M-SC       XY24       852536.0      6
H-SC       XR         798138.0      7
X-MC       X40        538738.0      7
           XY24       459814.0      3
           X40N       430733.0      4
X-LC       X40        418319.0      6
X-SC       X40N       319901.0      3
M-SC       OX40N      309395.0      5
H-SC       OX40N      303818.0      4
X-SC       XY24       300835.0      3
H-SC       SR         286123.0      2
H-MC       AR         238541.0      2
X-MC       XY25       228894.0      2
H-MC       XY24       207989.0      1
X-LC       X40N       206710.0      2
           XY24       205283.0      2
X-SC       X40        195102.0      2
M-SC       XR         190470.0      2
X-LC       XY25       135473.0      3
M-SC       AR         132192.0      2
M-LC       XR         105917.0      1
L-SC       OX40N       91341.0      1
           XR          87761.0      1
H-LC       AR          87571.0      1
L-MC       XR          48619.0      1
M-MC       XY25        40781.0      1
L-LC       XY25        20651.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
